In [1]:
using StatisticalRethinking, Distributed, JLD, LinearAlgebra
using Mamba

Data

In [2]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [3]:
df2 = filter(row -> row[:age] >= 18, df);

Input data for Mamba

In [4]:
data = Dict(
  :x => convert(Array{Float64,1}, df2[:weight]),
  :y => convert(Array{Float64,1}, df2[:height])
);

Log-transformed Posterior(b0, b1, log(s2)) + Constant and Gradient Vector

In [5]:
logfgrad = function(x::DenseVector)
  b0 = x[1]
  b1 = x[2]
  logs2 = x[3]
  r = data[:y] .- b0 .- b1 .* data[:x]
  logf = (-0.5 * length(data[:y]) - 0.001) * logs2 -
           (0.5 * dot(r, r) + 0.001) / exp(logs2) -
           0.5 * b0^2 / 1000 - 0.5 * b1^2 / 1000
  grad = [
    sum(r) / exp(logs2) - b0 / 1000,
    sum(data[:x] .* r) / exp(logs2) - b1 / 1000,
    -0.5 * length(data[:y]) - 0.001 + (0.5 * dot(r, r) + 0.001) / exp(logs2)
  ]
  logf, grad
end

# MCMC Simulation with No-U-Turn Sampling

n = 5000
burnin = 1000
sim = Mamba.Chains(n, 3, start = (burnin + 1), names = ["b0", "b1", "s2"])
theta = NUTSVariate([0.0, 0.0, 0.0], logfgrad)
for i in 1:n
  sample!(theta, adapt = (i <= burnin))
  if i > burnin
    sim[i, :, 1] = [theta[1:2]; exp(theta[3])]
  end
end

Summarize draws

In [6]:
describe(sim)#-

Iterations = 1001:5000
Thinning interval = 1
Chains = 1
Samples per chain = 4000

Empirical Posterior Estimates:
       Mean          SD       Naive SE       MCSE         ESS   
b0 113.64350329 1.875563283 0.0296552594 0.1008006105  346.20802
b1   0.91022225 0.041366315 0.0006540589 0.0021923347  356.02456
s2  25.97481892 1.977978952 0.0312745933 0.0495375234 1594.31727

Quantiles:
       2.5%         25.0%        50.0%       75.0%        97.5%   
b0 110.01219848 112.32160824 113.71651205 114.9162958 117.51249705
b1   0.82694344   0.88240065   0.90890542   0.9401661   0.99001366
s2  22.15664552  24.64460687  25.90624284  27.3046840  30.07624198



*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*